In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
import sys
import time
import warnings
warnings.filterwarnings("ignore")

### load FICO dataset

In [2]:
DATA_DIR = ''
file_name = ['transrisk_performance_by_race_ssa.csv','transrisk_cdf_by_race_ssa.csv','totals.csv']
df_repay = pd.read_csv(DATA_DIR + file_name[0])
df_cdf = pd.read_csv(DATA_DIR + file_name[1])
df_dm_ratio = pd.read_csv(DATA_DIR + file_name[2])

df_repay = df_repay.rename(columns={"Non- Hispanic white": "Caucasian", "Black": "African-American", "Hispanic":"Hispanic", "Asian":"Asian"})
df_cdf = df_cdf.rename(columns={"Non- Hispanic white": "Caucasian", "Black": "African-American", "Hispanic":"Hispanic", "Asian":"Asian"})
df_dm_ratio = df_dm_ratio.rename(columns={"Non- Hispanic white": "Caucasian", "Black": "African-American", "Hispanic":"Hispanic", "Asian":"Asian"})

df_cdf.iloc[:,1:] = df_cdf.iloc[:,1:]/100.0
df_repay.iloc[:,1:] = (100-df_repay.iloc[:,1:])/100.0
k1_list=np.linspace(1,2,5)

### obtain PDF from CDF

In [3]:
nrow, ncol = df_cdf.shape
df_pdf = df_cdf.copy()
for i in range(nrow-1):
    indx = i + 1
    df_pdf.iloc[i+1,1:] = df_cdf.iloc[i+1,1:]- df_cdf.iloc[i,1:]

In [ ]:
fig = plt.figure(figsize=(7,3))  
plt.subplot(1,2,1)
plt.plot(df_pdf["Score"]/100,df_pdf['Caucasian'],label='Caucasian')
plt.plot(df_pdf["Score"]/100,df_pdf['African-American'],label='African-American')
plt.plot(df_pdf["Score"]/100,df_pdf['Hispanic'],label='Hispanic')
plt.plot(df_pdf["Score"]/100,df_pdf['Asian'],label='Asian')
plt.xlabel('Score')
plt.title("PDF of scores")
plt.legend()
plt.subplot(1,2,2)
plt.plot(df_cdf["Score"]/100,df_cdf['Caucasian'],label='Caucasian')
plt.plot(df_cdf["Score"]/100,df_cdf['African-American'],label='African-American')
plt.plot(df_cdf["Score"]/100,df_cdf['Hispanic'],label='Hispanic')
plt.plot(df_cdf["Score"]/100,df_cdf['Asian'],label='Asian')
plt.xlabel('Score')
plt.title("CDF of scores")
plt.tight_layout()
fig.savefig('pdf_cdf.pdf')
plt.show()

### repayment probability $P_{Y|X,S}(1|x,s)$

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 10.7})

fig = plt.figure(figsize=(3.5,3))  
plt.plot(df_repay["Score"]/100,df_repay.iloc[:,1],label='Caucasian')
plt.plot(df_repay["Score"]/100,df_repay.iloc[:,2],label='African-American')
plt.plot(df_repay["Score"]/100,df_repay.iloc[:,3],label='Hispanic')
plt.plot(df_repay["Score"]/100,df_repay.iloc[:,4],label='Asian')
plt.title(r"$P_{Y|X,S}(1|x,s)$")
plt.xlabel('Score')
plt.legend()
plt.tight_layout()
fig.savefig('repay_prob.pdf')
plt.show()

### Create a simulator

**Step 1.** Generate simulated data from the joint distributions given above, $P(X=x,Y=y \mid S= s) = P(Y=y \mid X=x, S= s)P(X=x\mid S=s)$.

In [4]:
np.random.seed(777)
# Step 1
NUM_SAMPLES = 100000
elements = df_pdf["Score"]
probabilities_c = df_pdf["Caucasian"]
probabilities_aa = df_pdf["African-American"]
probabilities_h = df_pdf["Hispanic"]
probabilities_a = df_pdf["Asian"]

scores_c = np.random.choice(elements, NUM_SAMPLES, p=probabilities_c)
scores_aa = np.random.choice(elements, NUM_SAMPLES, p=probabilities_aa)
scores_h = np.random.choice(elements, NUM_SAMPLES, p=probabilities_h)
scores_a = np.random.choice(elements, NUM_SAMPLES, p=probabilities_a)
prob_repay_c = [df_repay.loc[df_repay["Score"]==x]["Caucasian"].values[0] for x in scores_c]
prob_repay_aa = [df_repay.loc[df_repay["Score"]==x]["African-American"].values[0] for x in scores_aa]
prob_repay_h = [df_repay.loc[df_repay["Score"]==x]["Hispanic"].values[0] for x in scores_h]
prob_repay_a = [df_repay.loc[df_repay["Score"]==x]["Asian"].values[0] for x in scores_a]

repay_c = np.random.binomial(1, prob_repay_c, NUM_SAMPLES)
repay_aa = np.random.binomial(1, prob_repay_aa, NUM_SAMPLES)
repay_h = np.random.binomial(1, prob_repay_h, NUM_SAMPLES)
repay_a = np.random.binomial(1, prob_repay_a, NUM_SAMPLES)

scores_c = scores_c/100.0
scores_aa = scores_aa/100.0
scores_h = scores_h/100.0
scores_a = scores_a/100.0

**Step 2.** Fit beta distributions to the generated data

In [ ]:
from scipy.stats import beta
import scipy.stats as ss
# Step 2
a_c1,b_c1,loc_c1,scale_c1 = beta.fit(scores_c[repay_c==1])
a_c0,b_c0,loc_c0,scale_c0 = beta.fit(scores_c[repay_c==0])
a_aa0,b_aa0,loc_aa0,scale_aa0 = beta.fit(scores_aa[repay_aa==0])
a_aa1,b_aa1,loc_aa1,scale_aa1 = beta.fit(scores_aa[repay_aa==1])

a_h0,b_h0,loc_h0,scale_h0 = beta.fit(scores_h[repay_h==0])
a_h1,b_h1,loc_h1,scale_h1 = beta.fit(scores_h[repay_h==1])
a_a0,b_a0,loc_a0,scale_a0 = beta.fit(scores_a[repay_a==0])
a_a1,b_a1,loc_a1,scale_a1 = beta.fit(scores_a[repay_a==1])


def plot_beta(x_range, a, b, mu=0, sigma=1, cdf=False, **kwargs):
    '''
    Plots the f distribution function for a given x range, a and b
    If mu and sigma are not provided, standard beta is plotted
    If cdf=True cumulative distribution is plotted
    Passes any keyword arguments to matplotlib plot function
    '''
    x = x_range
    if cdf:
        y = ss.beta.cdf(x, a, b, mu, sigma)
    else:
        y = ss.beta.pdf(x, a, b, mu, sigma)
    plt.plot(x, y, **kwargs)

x = np.linspace(0, 1, 5000)
fig = plt.figure(figsize=(12, 2.5))
plt.subplot(1,4,1)
plt.ylim(0.1, 11.6)
plt.xlim(0, 1)
plot_beta(x, a_c1,b_c1, 0, 1, color='green', lw=2, ls='-')
plot_beta(x, a_c0,b_c0, 0, 1, color='red', lw=2, ls='-')
plot_beta(x,(a_c0+a_c1)/2,(b_c0+b_c1)/2, 0, 1, color='blue', label = 'Fail to improve', lw=2, ls='-')
plt.hist(scores_c[repay_c==1], density=True, color = 'green', bins=100, label='repay',alpha=0.5)
plt.hist(scores_c[repay_c==0], density=True, bins=100, color = 'red', label='default',alpha=0.5)
plt.title("Caucasian")
plt.xlabel('Score')
plt.legend()
plt.subplot(1,4,2)
plt.ylim(0.1, 11.6)
plt.xlim(0, 1)
plot_beta(x, a_aa1,b_aa1, 0, 1, color='green', lw=2, ls='-')
plot_beta(x,a_aa0,b_aa0, 0, 1, color='red', lw=2, ls='-')
plot_beta(x,(a_aa0+a_aa1)/2,(b_aa0+b_aa1)/2, 0, 1, color='blue', label = 'Fail to improve', lw=2, ls='-')
plt.hist(scores_aa[repay_aa==1], density=True, color = 'green', bins=100, label='repay',alpha=0.5)
plt.hist(scores_aa[repay_aa==0], density=True, color = 'red', bins=100, label='default',alpha=0.5)
plt.title("African-American")
plt.xlabel('Score')
plt.legend()
plt.subplot(1,4,3)
plt.ylim(0.1, 11.6)
plt.xlim(0, 1)
plot_beta(x, a_h1,b_h1, 0, 1, color='green', lw=2, ls='-')
plot_beta(x,a_h0,b_h0, 0, 1, color='red', lw=2, ls='-')
plot_beta(x,(a_h0+a_h1)/2,(b_h0+b_h1)/2, 0, 1, color='blue', label = 'Fail to improve', lw=2, ls='-')
plt.hist(scores_h[repay_h==1], density=True, bins=100, color = 'green', label='repay',alpha=0.5)
plt.hist(scores_h[repay_h==0], density=True, bins=100, color = 'red', label='default',alpha=0.5)
plt.title("Hispanic")
plt.xlabel('Score')
plt.legend()
plt.subplot(1,4,4)
plt.ylim(0.1, 11.6)
plt.xlim(0, 1)
plot_beta(x, a_a1,b_a1, 0, 1, color='green', lw=2, ls='-')
plot_beta(x,a_a0,b_a0, 0, 1, color='red', lw=2, ls='-')
plot_beta(x,(a_a0+a_a1)/2,(b_a0+b_a1)/2, 0, 1, color='blue', label = 'Fail to improve', lw=2, ls='-')
plt.hist(scores_a[repay_a==1], density=True, bins=100, color = 'green', label='repay',alpha=0.5)
plt.hist(scores_a[repay_a==0], density=True, bins=100, color = 'red', label='default',alpha=0.5)
plt.title("Asian")
plt.xlabel('Score')
plt.legend()
plt.tight_layout()
fig.savefig('score_pdf.pdf')
plt.show()

**Assume $$P^I(\theta) \sim Beta(\frac{a_0+a_1}{2}, \frac{b_0+b_1}{2}, \mu, \sigma)$$**



In [9]:
from matplotlib.ticker import LogLocator
import warnings
warnings.filterwarnings("ignore")

def plot_monotone_1(x_range, a0, b0, a1, b1, mu=0, sigma=1, **kwargs):
    '''
    Plots the f distribution function for a given x range, a and b
    If mu and sigma are not provided, standard beta is plotted
    If cdf=True cumulative distribution is plotted
    Passes any keyword arguments to matplotlib plot function
    '''
    x = x_range
    y0 = ss.beta.pdf(x, a0, b0, mu, sigma)
    y1 = ss.beta.pdf(x, a1, b1, mu, sigma)
    ratio_10 = y1/y0
    plt.plot(x, ratio_10, **kwargs)
    plt.yscale('log')

def plot_monotone_2(x_range, a0, b0, a1, b1, mu=0, sigma=1, **kwargs):
    '''
    Plots the f distribution function for a given x range, a and b
    If mu and sigma are not provided, standard beta is plotted
    If cdf=True cumulative distribution is plotted
    Passes any keyword arguments to matplotlib plot function
    '''
    x = x_range
    yi = ss.beta.pdf(x, (a0+a1)/2, (b0+b1)/2, mu, sigma)
    y1 = ss.beta.pdf(x, a1, b1, mu, sigma)
    ratio_1i = y1/yi
    plt.plot(x, ratio_1i, **kwargs)
    plt.yscale('log')   
    
def plot_monotone_3(x_range, a0, b0, a1, b1, mu=0, sigma=1, **kwargs):
    '''
    Plots the f distribution function for a given x range, a and b
    If mu and sigma are not provided, standard beta is plotted
    If cdf=True cumulative distribution is plotted
    Passes any keyword arguments to matplotlib plot function
    '''
    x = x_range
    yi = ss.beta.pdf(x, (a0+a1)/2, (b0+b1)/2, mu, sigma)
    y0 = ss.beta.pdf(x, a0, b0, mu, sigma)
    ratio_i0 = yi/y0
    plt.plot(x, ratio_i0, **kwargs)
    plt.yscale('log')  

In [ ]:
fig = plt.figure(figsize=(3.5, 3))   
ax = fig.add_subplot(111)
plt.xlim(0, 1)
plt.ylim(0.00001, 10**14)
plot_monotone_1(x,a_c0,b_c0,a_c1,b_c1, 0, 1, lw=1.5, ls='-',label='Caucasian')
plot_monotone_1(x,a_aa0,b_aa0,a_aa1,b_aa1, 0, 1, lw=1.5, ls='-',label='African-American')
plot_monotone_1(x,a_h0,b_h0,a_h1,b_h1, 0, 1,  lw=1.5, ls='-',label='Hispanic')
plot_monotone_1(x,a_a0,b_a0,a_a1,b_a1, 0, 1, lw=1.5, ls='-',label='Asian')

ax.yaxis.set_major_locator(LogLocator(base=10**13))

plt.xlabel('Score')
plt.title(r'$\frac{P_{X|Y,S}(x|1,s)}{P_{X|Y,S}(x|0,s)}$')
plt.legend()
plt.tight_layout()
fig.savefig('p10.pdf')
plt.show()

In [ ]:
fig = plt.figure(figsize=(3.5, 3))   
ax = fig.add_subplot(111)
plt.xlim(0, 1)
plt.ylim(0.00001, 10**14)
plot_monotone_2(x,a_c0,b_c0,a_c1,b_c1, 0, 1, lw=1.5, ls='-',label='Caucasian')
plot_monotone_2(x,a_aa0,b_aa0,a_aa1,b_aa1, 0, 1, lw=1.5, ls='-',label='African-American')
plot_monotone_2(x,a_h0,b_h0,a_h1,b_h1, 0, 1,  lw=1.5, ls='-',label='Hispanic')
plot_monotone_2(x,a_a0,b_a0,a_a1,b_a1, 0, 1, lw=1.5, ls='-',label='Asian')

ax.yaxis.set_major_locator(LogLocator(base=10**13))

plt.xlabel('Score')
plt.title(r'$\frac{P_{X|Y,S}(x|1,s)}{P^I(x)}$')
plt.legend()
plt.tight_layout()
fig.savefig('p1i.pdf')
plt.show()

In [ ]:
fig = plt.figure(figsize=(3.5, 3))   
ax = fig.add_subplot(111)
plt.xlim(0, 1)
plt.ylim(0.00001, 10**14)
plot_monotone_3(x,a_c0,b_c0,a_c1,b_c1, 0, 1, lw=1.5, ls='-',label='Caucasian')
plot_monotone_3(x,a_aa0,b_aa0,a_aa1,b_aa1, 0, 1, lw=1.5, ls='-',label='African-American')
plot_monotone_3(x,a_h0,b_h0,a_h1,b_h1, 0, 1,  lw=1.5, ls='-',label='Hispanic')
plot_monotone_3(x,a_a0,b_a0,a_a1,b_a1, 0, 1, lw=1.5, ls='-',label='Asian')

ax.yaxis.set_major_locator(LogLocator(base=10**13))

plt.xlabel('Score')
plt.title(r'$\frac{P^I(x)}{P_{X|Y,S}(x|0,s)}$')
plt.legend()
plt.tight_layout()
fig.savefig('pi0.pdf')
plt.show()

#### demographic parameters

The proportion of each demographic group

In [ ]:
# relative sizes
total = df_dm_ratio.iloc[0]['Caucasian']+df_dm_ratio.iloc[0]['African-American']+df_dm_ratio.iloc[0]['Hispanic']+df_dm_ratio.iloc[0]['Asian']
P_c = df_dm_ratio.iloc[0]['Caucasian']/total
P_aa = df_dm_ratio.iloc[0]['African-American']/total
P_h = df_dm_ratio.iloc[0]['Hispanic']/total
P_a = df_dm_ratio.iloc[0]['Asian']/total
print(P_c,P_aa,P_h,P_a)

$\alpha_s$

In [ ]:
# repayment rates
alpha_aa, alpha_c,alpha_h, alpha_asian = np.sum(repay_aa)/NUM_SAMPLES, np.sum(repay_c)/NUM_SAMPLES,np.sum(repay_h)/NUM_SAMPLES, np.sum(repay_a)/NUM_SAMPLES
print(alpha_aa, alpha_c,alpha_h, alpha_asian)

## Verification of results

We do experiments on:

1. Caucasian vs African American

2. Asian vs Hispanic

### 1. (Non)-strategic optimal threshold and utility

In [5]:
from scipy.stats import norm
eps = 0.5
u = 1
q = 0.3
mu_c = 0
std_c = 0.25

In [6]:
# scaled utility function & non-strategic optimal threshold 
def opt_non_strategic(u,alpha,a0,b0,a1,b1):
    theta = np.linspace(0,1,10000)
    U = (1 - beta.cdf(theta,a1,b1))*alpha*u - (1-beta.cdf(theta,a0,b0))*(1-alpha)*u  
    return U, theta[int(np.argmax(U))]
    
# one group: measure for calculating unfairness
def fair_measure(threshold,a0,b0,a1,b1,alpha,fair_type):
    if fair_type == "eqopt":
        return beta.cdf(threshold, a1, b1, 0, 1)
    if fair_type == "dp":
        return beta.cdf(threshold, a1, b1, 0, 1)*alpha + beta.cdf(threshold, a0, b0, 0, 1)*(1-alpha)

# Get manipulation Prob
def get_manip_prob(theta, q, eps, ai, bi, a1, b1):
    """
    function to get manip prob under q and eps
    """
    value = (1-q)*(beta.cdf(theta, ai, bi, 0, 1) - beta.cdf(theta, a1, b1, 0, 1)) - eps*(1 - beta.cdf(theta, a1, b1, 0, 1))
    return norm.cdf(value, mu_c, std_c)   

    
# Get Phi, which is strategic - non-strategic
def get_Phi(u, alpha, theta, q, eps, a0, b0, ai, bi, a1, b1, k1 = 1, k2 = 1, k3 = 1):
    """
    Get Phi, which is strategic - non-strategic
    k1, k2, k3 measure our preferences for 3 kinds of behaviors
    larger k1 means we prefer improvement
    larger k2 means we do not prefer failure of improvement
    larger k3 means we do not prefer manipulation
    k1=k2=k3=1 is the naive difference
    """
    # manipulation probability
    pm = get_manip_prob(theta, q, eps, ai, bi, a1, b1)
    # Reverse Benefit
    b_1 = k1*(1-pm)*q*(2 - beta.cdf(theta, a0, b0, 0, 1) - beta.cdf(theta, a1, b1, 0, 1))
   
    # Loss of Failure of Improvement 
    l_1 = k2*(1-pm)*(1-q)*(beta.cdf(theta, a0, b0, 0, 1) - beta.cdf(theta, ai, bi, 0, 1))
    # Loss of manipulation
    
    l_2 = k3*pm*((1-eps)*(1 - beta.cdf(theta, a1, b1, 0, 1))-(1-beta.cdf(theta, a0, b0, 0, 1)))
    
    return u*(1-alpha)*(b_1-l_1-l_2)
    
def get_strat_utility(u, alpha, theta, q, eps, a0, b0, ai, bi, a1, b1, k1 = 1, k2 = 1, k3 = 1):
    theta_grid, k1_grid = np.meshgrid(theta, k1, indexing='ij')
    def comp(tta, k_1):
        phi = get_Phi(u, alpha, tta, q, eps, a0, b0, ai, bi, a1, b1, k_1, k2, k3)
        U = (1 - beta.cdf(tta,a1,b1))*alpha*u - (1 - beta.cdf(tta,a0,b0))*(1-alpha)*u + phi
        return U
    vectorized_compute_U = np.vectorize(comp)
    U_values = vectorized_compute_U(theta_grid, k1_grid)
    max_U_indices = np.argmax(U_values, axis=0)
    theta_max_U = theta_grid[max_U_indices, np.arange(len(k1_list))]
    max_U_values = U_values[max_U_indices, np.arange(len(k1_list))]
    
    return max_U_values, theta_max_U


def get_strat_optimal(u, alpha, q, eps, a0, b0, ai, bi, a1, b1, k1=1, k2=1, k3=1):
    theta = np.linspace(0,1,1000)
    return get_strat_utility(u = u, alpha = alpha, theta = theta, q = q, eps = eps, a0 = a0, b0 = b0, ai = ai, bi = bi, a1 = a1, b1 = b1, k1 = k1, k2 = k2, k3 = k3)

In [ ]:
print(a_c0,b_c0,a_c1,b_c1)
print(a_aa0,b_aa0,a_aa1,b_aa1)
print(a_h0,b_h0,a_h1,b_h1)
print(a_a0,b_a0,a_a1,b_a1)

**(1)**. Threshold for Caucasian and African American

**Caucasian**

In [19]:
def vary_eps(u, alpha, a0, b0, ai, bi, a1, b1, k1 = 1, k2 = 1, k3 = 1, q_list = [0,0.3,0.6,1], eps_list = np.linspace(0,1,100), ):
    opt_no = opt_non_strategic(u,alpha,a0,b0,a1,b1)[1]
    for i in range(len(q_list)):
        q = q_list[i]
        x = np.zeros(100)
        ul = np.zeros(100)
        m = np.zeros(100)
        j = 0
        for eps in eps_list:
            u1, x1 = get_strat_optimal(u, alpha, q, eps, a0, b0, ai, bi, a1, b1, k1, k2, k3)
            u1 = u1[int(np.argmax(u1))]
            x[j] = x1
            ul[j] = u1
            m[j] = get_manip_prob(x1, q, eps, ai, bi, a1, b1)
            j += 1
        df = pd.DataFrame({'eps': np.linspace(0,1,100), 'theta_max': x, 'max utility': ul, 'manipulation prob': m})
        axes[int(i/2)][int(i%2)].tick_params(axis='both', which='major', labelsize=12)
        axes[int(i/2)][int(i%2)].plot(df['eps'], df['theta_max'],'r-', alpha=0.6, label='strategic optimal')
        axes[int(i/2)][int(i%2)].plot(np.linspace(0,1,100), [opt_no]*100, 'b-', label = 'non-strategic optimal')
        axes[int(i/2)][int(i%2)].set_xlabel(r'$\epsilon$',fontsize = 12)
        axes[int(i/2)][int(i%2)].set_ylabel(r'$\theta$',fontsize=12)
        axes[int(i/2)][int(i%2)].legend(fontsize = 11)
        axes[int(i/2)][int(i%2)].set_title('q = %.2f'%q, fontsize=11)

In [ ]:
fig, axes = plt.subplots(2,2)
fig.set_size_inches(8, 6)
vary_eps(u, alpha_c, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
plt.tight_layout()
fig.savefig('Caucasian.pdf')

**African American**

In [ ]:
fig, axes = plt.subplots(2,2)
fig.set_size_inches(8, 6)
vary_eps(u, alpha_aa, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
plt.tight_layout()
fig.savefig('AA.pdf')

**(2) Effects of varying k_1, k_2** 

Plot the manipulation probability

In [20]:
uhat_c = opt_non_strategic(u,alpha_c,a_c0,b_c0,a_c1,b_c1)[1]
uhat_aa = opt_non_strategic(u,alpha_aa,a_aa0,b_aa0,a_aa1,b_aa1)[1]
theta_c_list, _ = get_strat_optimal(u, alpha_c, q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1, k1_list)
strat_func = np.vectorize(get_strat_optimal, excluded=['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k2', 'k3'])
theta_aa_list, _ = get_strat_optimal(u, alpha_aa, q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1, k1_list)
u_c = theta_c_list[0]
u_aa = theta_aa_list[0]
q = 0.3
eps = 0.5

In [ ]:
# plot manipulation probability
x = np.linspace(0,1,100)
plt.figure(figsize = (4,3))
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
prob1 = get_manip_prob(x,q,eps,(a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
prob2 = get_manip_prob(x,q, eps, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
plt.plot(x,prob1,label = r'$P_M$ (Caucasian)')
plt.plot(x,prob2,label = r'$P_M$ (African American)')
prob_a = get_manip_prob(u_c, q, eps, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
prob_b = get_manip_prob(u_aa,q, eps, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
plt.plot(u_c, prob_a,  marker="o", markersize=10, color = 'orange',label = r'$P_M(\theta_c^*)$')
plt.plot(u_aa, prob_b, marker="o", markersize=10, color = 'green',label = r'$P_M(\theta_{aa}^*)$')
plt.xlabel(r'$\theta$',fontsize = 14)
plt.ylabel(r'$P_M(\theta^*)$',fontsize = 14)
plt.legend()
plt.tight_layout()
plt.savefig('manipcurve_c_aa.pdf')

In [21]:
def non_strat_utility(u,alpha,theta,a0,b0,a1,b1):
    U = (1 - beta.cdf(theta,a1,b1))*alpha*u - (1-beta.cdf(theta,a0,b0))*(1-alpha)*u  
    return U

k1_list = np.linspace(1,2,5)
_, theta_c_list = get_strat_optimal(u, alpha_c, q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1, k1_list)
phi_func = np.vectorize(get_Phi, excluded = ['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k1','k2','k3'])
nonstrat_func = np.vectorize(non_strat_utility, excluded = ['u','alpha','a0','b0','a1','b1'])
utility_c_list = phi_func(u, alpha_c, theta_c_list, q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1) + nonstrat_func(u, alpha_c, theta_c_list, a_c0, b_c0,a_c1, b_c1)
utility_c_non = [get_Phi(u, alpha_c, uhat_c, q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1) + non_strat_utility(u, alpha_c, uhat_c, a_c0, b_c0,a_c1, b_c1)]*5
manip_list_c = get_manip_prob(theta_c_list,q,eps,(a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
cdf_c = np.array([fair_measure(theta, a_c0, b_c0, a_c1, b_c1, alpha_c, 'dp') for theta in theta_c_list])
manip_hat_c = get_manip_prob(uhat_c,q,eps,(a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
cdf_c_hat = fair_measure(uhat_c, a_c0, b_c0, a_c1, b_c1, alpha_c, 'dp')

In [ ]:
theta_c_list

In [116]:
k1_list = np.linspace(1,2,5)
strat_func = np.vectorize(get_strat_optimal, excluded=['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k2', 'k3'])
_, theta_aa_list = get_strat_optimal(u, alpha_aa, q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1, k1_list)
phi_func = np.vectorize(get_Phi, excluded = ['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k1','k2','k3'])
nonstrat_func = np.vectorize(non_strat_utility, excluded = ['u','alpha','a0','b0','a1','b1'])
utility_aa_list = phi_func(u, alpha_aa, theta_aa_list, q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1) + nonstrat_func(u, alpha_aa, theta_aa_list, a_aa0, b_aa0,a_aa1, b_aa1)
utility_aa_non = [get_Phi(u, alpha_aa, uhat_aa, q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1) + non_strat_utility(u, alpha_aa, uhat_aa, a_aa0, b_aa0,a_aa1, b_aa1)]*5
manip_list_aa = get_manip_prob(theta_aa_list,q,eps,(a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
cdf_aa = np.array([fair_measure(theta, a_aa0, b_aa0, a_aa1, b_aa1, alpha_aa, 'dp') for theta in theta_aa_list])
manip_hat_aa = get_manip_prob(uhat_aa,q,eps,(a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
cdf_aa_hat = fair_measure(uhat_aa, a_aa0, b_aa0, a_aa1, b_aa1, alpha_aa, 'dp')

In [ ]:
# can output every metric
# example for unfairness
cdf_aa - cdf_c

In [ ]:
# plot optimal threshold, (non)-strategic utility, and manipulation probability
x = k1_list
fig, axes = plt.subplots(2,1)
fig.set_size_inches(4, 7)
axes[0].tick_params(axis='both', which='major', labelsize=14)
axes[1].tick_params(axis='both', which='major', labelsize=14)
axes[0].plot(x,manip_list_c, marker = 'o',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
axes[0].plot(x,utility_c_list,marker = 'o',label = r'strategic utility',linestyle = 'solid',color = 'blue')
axes[0].plot(x,cdf_c, marker = 'o',label = r'$F_{X|S}(\theta|s)$',linestyle = 'dotted',color = 'green')
axes[0].plot(x, utility_c_non, marker = 'o', label = r'non-strategic utility',linestyle = 'solid',color = 'red')
axes[0].plot(x, [manip_hat_c]*5,marker = 'o',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
# axes[0].plot(x, [cdf_c_hat]*5,label = r'$F_{X|Y,S}(\widehat{theta}^*|1,s)$')
axes[0].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$', fontsize = 12)
axes[0].set_ylabel(r'value', fontsize = 12)
axes[0].legend(fontsize = 10)
axes[0].set_title('Caucasian',fontsize = 12)

axes[1].plot(x,manip_list_aa, marker = 'o',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
axes[1].plot(x,utility_aa_list,marker = 'o',label = r'strategic utility',linestyle = 'solid',color = 'blue')
axes[1].plot(x,cdf_aa, marker = 'o',label = r'$F_{X|S}(\theta|s)$',linestyle = 'dotted',color = 'green')
axes[1].plot(x, utility_aa_non ,marker = 'o',label = r'non-strategic utility',linestyle = 'solid',color = 'red')
axes[1].plot(x, [manip_hat_aa]*5,marker = 'o',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
# axes[1].plot(x, [cdf_aa_hat]*5,label = r'$F_{X|Y,S}(\widehat{\theta}^*|1,s)$')
axes[1].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$', fontsize = 12)
axes[1].set_ylabel(r'value', fontsize = 12)
axes[1].legend(fontsize = 10)
axes[1].set_title('African American',fontsize = 12)
plt.tight_layout()
fig.savefig('fairness_caa_dp.pdf')

**(2) Asian and Hispanic**

In [23]:
# scaled utility function & non-strategic optimal threshold 
def opt_non_strategic(u,alpha,a0,b0,a1,b1):
    theta = np.linspace(0,1,10000)
    U = (1 - beta.cdf(theta,a1,b1))*alpha*u - (1-beta.cdf(theta,a0,b0))*(1-alpha)*u  
    return U, theta[int(np.argmax(U))]
    
# one group: measure for calculating unfairness
def fair_measure(threshold,a0,b0,a1,b1,alpha,fair_type):
    if fair_type == "eqopt":
        return beta.cdf(threshold, a1, b1, 0, 1)
    if fair_type == "dp":
        return beta.cdf(threshold, a1, b1, 0, 1)*alpha + beta.cdf(threshold, a0, b0, 0, 1)*(1-alpha)

# Get manipulation Prob
def get_manip_prob(theta, q, eps, ai, bi, a1, b1):
    """
    function to get manip prob under q and eps
    """
    value = (1-q)*(beta.cdf(theta, ai, bi, 0, 1) - beta.cdf(theta, a1, b1, 0, 1)) - eps*(1 - beta.cdf(theta, a1, b1, 0, 1))
    return norm.cdf(value, mu_c, std_c)   

    
# Get Phi, which is strategic - non-strategic
def get_Phi(u, alpha, theta, q, eps, a0, b0, ai, bi, a1, b1, k1 = 1, k2 = 1, k3 = 1):
    """
    Get Phi, which is strategic - non-strategic
    k1, k2, k3 measure our preferences for 3 kinds of behaviors
    larger k1 means we prefer improvement
    larger k2 means we do not prefer failure of improvement
    larger k3 means we do not prefer manipulation
    k1=k2=k3=1 is the naive difference
    """
    # manipulation probability
    pm = get_manip_prob(theta, q, eps, ai, bi, a1, b1)
    # Reverse Benefit
    b_1 = k1*(1-pm)*q*(2 - beta.cdf(theta, a0, b0, 0, 1) - beta.cdf(theta, a1, b1, 0, 1))
   
    # Loss of Failure of Improvement 
    l_1 = k2*(1-pm)*(1-q)*(beta.cdf(theta, a0, b0, 0, 1) - beta.cdf(theta, ai, bi, 0, 1))
    # Loss of manipulation
    
    l_2 = k3*pm*((1-eps)*(1 - beta.cdf(theta, a1, b1, 0, 1))-(1-beta.cdf(theta, a0, b0, 0, 1)))
    
    return u*(1-alpha)*(b_1-l_1-l_2)
    
def get_strat_utility(u, alpha, theta, q, eps, a0, b0, ai, bi, a1, b1, k1 = 1, k2 = 1, k3 = 1):
    phi = get_Phi(u, alpha, theta, q, eps, a0, b0, ai, bi, a1, b1, k1, k2, k3)
    U = (1 - beta.cdf(theta,a1,b1))*alpha*u - (1 - beta.cdf(theta,a0,b0))*(1-alpha)*u + phi
    return U

def get_strat_optimal(u, alpha, q, eps, a0, b0, ai, bi, a1, b1, k1=1, k2=1, k3=1):
    theta = np.linspace(0,1,10000)
    U = get_strat_utility(u = u, alpha = alpha, theta = theta, q = q, eps = eps, a0 = a0, b0 = b0, ai = ai, bi = bi, a1 = a1, b1 = b1, k1 = k1, k2 = k2, k3 = k3)
    return U, theta[int(np.argmax(U))]

In [ ]:
fig, axes = plt.subplots(2,2)
fig.set_size_inches(8, 6)
vary_eps(u, alpha_asian, a_a0, b_a0, (a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1)
plt.tight_layout()
fig.savefig('Asian.pdf')

In [ ]:
fig, axes = plt.subplots(2,2)
fig.set_size_inches(8, 6)
vary_eps(u, alpha_h, a_h0, b_h0, (a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1)
plt.tight_layout()
fig.savefig('Hispanic.pdf')

In [24]:
def get_strat_utility(u, alpha, theta, q, eps, a0, b0, ai, bi, a1, b1, k1 = 1, k2 = 1, k3 = 1):
    theta_grid, k1_grid = np.meshgrid(theta, k1, indexing='ij')
    def comp(tta, k_1):
        phi = get_Phi(u, alpha, tta, q, eps, a0, b0, ai, bi, a1, b1, k_1, k2, k3)
        U = (1 - beta.cdf(tta,a1,b1))*alpha*u - (1 - beta.cdf(tta,a0,b0))*(1-alpha)*u + phi
        return U
    vectorized_compute_U = np.vectorize(comp)
    U_values = vectorized_compute_U(theta_grid, k1_grid)
    max_U_indices = np.argmax(U_values, axis=0)
    theta_max_U = theta_grid[max_U_indices, np.arange(len(k1_list))]
    max_U_values = U_values[max_U_indices, np.arange(len(k1_list))]
    
    return max_U_values, theta_max_U


def get_strat_optimal(u, alpha, q, eps, a0, b0, ai, bi, a1, b1, k1=1, k2=1, k3=1):
    theta = np.linspace(0,1,1000)
    return get_strat_utility(u = u, alpha = alpha, theta = theta, q = q, eps = eps, a0 = a0, b0 = b0, ai = ai, bi = bi, a1 = a1, b1 = b1, k1 = k1, k2 = k2, k3 = k3)

uhat_a = opt_non_strategic(u,alpha_asian,a_a0,b_a0,a_a1,b_a1)[1]
uhat_h = opt_non_strategic(u,alpha_h,a_h0,b_h0,a_h1,b_h1)[1]
theta_a_list, _ = get_strat_optimal(u, alpha_asian, q, eps, a_a0, b_a0, (a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1, k1_list)
strat_func = np.vectorize(get_strat_optimal, excluded=['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k2', 'k3'])
theta_h_list, _ = get_strat_optimal(u, alpha_h, q, eps, a_h0, b_h0, (a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1, k1_list)
u_a = theta_a_list[0]
u_h = theta_h_list[0]
q = 0.3
eps = 0.5

In [ ]:
# plot manipulation probability
x = np.linspace(0,1,100)
plt.figure(figsize = (4,3))
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
prob1 = get_manip_prob(x,q,eps,(a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1)
prob2 = get_manip_prob(x,q, eps, (a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1)
plt.plot(x,prob1,label = r'$P_M$ curve for Asian')
plt.plot(x,prob2,label = r'$P_M$ curve for Hispanic')
prob_a = get_manip_prob(u_a, q, eps, (a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1)
prob_b = get_manip_prob(u_h,q, eps, (a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1)
plt.plot(u_a, prob_a,  marker="o", markersize=10, label = r'$P_M(\theta_a^*)$')
plt.plot(u_h, prob_b, marker="o", markersize=10, label = r'$P_M(\theta_h^*)$')
plt.xlabel(r'$\theta$',fontsize = 14)
plt.ylabel(r'$P_M(\theta^*)$',fontsize = 14)
plt.legend()
plt.tight_layout()
plt.savefig('manipcurve_a_h.pdf')

In [38]:

k1_list = np.linspace(1,2,5)
_, theta_a_list = get_strat_optimal(u, alpha_asian, q, eps, a_a0, b_a0, (a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1, k1_list)
phi_func = np.vectorize(get_Phi, excluded = ['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k1','k2','k3'])
nonstrat_func = np.vectorize(non_strat_utility, excluded = ['u','alpha','a0','b0','a1','b1'])
utility_a_list = phi_func(u, alpha_asian, theta_a_list, q, eps, a_a0, b_a0, (a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1) + nonstrat_func(u, alpha_asian, theta_a_list, a_a0, b_a0,a_a1, b_a1)
utility_a_non = [get_Phi(u, alpha_asian, uhat_a, q, eps, a_a0, b_a0, (a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1) + non_strat_utility(u, alpha_asian, uhat_a, a_a0, b_a0,a_a1, b_a1)]*5
manip_list_a = get_manip_prob(theta_a_list,q,eps,(a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1)
cdf_a = np.array([fair_measure(theta, a_a0, b_a0, a_a1, b_a1, alpha_asian, 'dp') for theta in theta_a_list])
manip_hat_a = get_manip_prob(uhat_a,q,eps,(a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1)
cdf_a_hat = fair_measure(uhat_a, a_a0, b_a0, a_a1, b_a1, alpha_asian, 'dp')

In [39]:
k1_list = np.linspace(1,2,5)
_, theta_h_list = get_strat_optimal(u, alpha_h, q, eps, a_h0, b_h0, (a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1, k1_list)
phi_func = np.vectorize(get_Phi, excluded = ['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k1','k2','k3'])
nonstrat_func = np.vectorize(non_strat_utility, excluded = ['u','alpha','a0','b0','a1','b1'])
utility_h_list = phi_func(u, alpha_h, theta_h_list, q, eps, a_h0, b_h0, (a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1) + nonstrat_func(u, alpha_h, theta_h_list, a_h0, b_h0,a_h1, b_h1)
utility_h_non = [get_Phi(u, alpha_h, uhat_h, q, eps, a_h0, b_h0, (a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1) + non_strat_utility(u, alpha_h, uhat_h, a_h0, b_h0,a_h1, b_h1)]*5
manip_list_h = get_manip_prob(theta_h_list,q,eps,(a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1)
cdf_h = np.array([fair_measure(theta, a_h0, b_h0, a_h1, b_h1, alpha_h, 'dp') for theta in theta_h_list])
manip_hat_h = get_manip_prob(uhat_h,q,eps,(a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1)
cdf_h_hat = fair_measure(uhat_h, a_h0, b_h0, a_h1, b_h1, alpha_h, 'dp')

In [ ]:
# plot optimal threshold, (non)-strategic utility, and manipulation probability
x = k1_list
fig, axes = plt.subplots(2,1)
fig.set_size_inches(4, 7)
axes[0].tick_params(axis='both', which='major')
axes[1].tick_params(axis='both', which='major')
axes[0].plot(x,manip_list_a, marker = 'o',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
axes[0].plot(x,utility_a_list,marker = 'o',label = r'strategic utility',linestyle = 'solid',color = 'blue')
axes[0].plot(x,cdf_a, marker = 'o',label = r'$F_{X|Y,S}(\theta|1,s)$',linestyle = 'dotted',color = 'green')
axes[0].plot(x, utility_a_non, marker = 'o',label = r'non-strategic utility',linestyle = 'solid',color = 'red')
axes[0].plot(x, [manip_hat_a]*5,marker = 'o',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
axes[0].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$',fontsize = 12)
axes[0].set_ylabel(r'value',fontsize = 12)
axes[0].legend()
axes[0].set_title('Asian')

axes[1].plot(x,manip_list_h, marker = 'o',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
axes[1].plot(x,utility_h_list,marker = 'o',label = r'strategic utility',linestyle = 'solid',color = 'blue')
axes[1].plot(x,cdf_h, marker = 'o',label = r'$F_{X|Y,S}(\theta|1,s)$',linestyle = 'dotted',color = 'green')
axes[1].plot(x, utility_h_non, marker= 'o', label = r'non-strategic utility',linestyle = 'solid',color = 'red')
axes[1].plot(x, [manip_hat_h]*5,marker = 'o',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
axes[1].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$',fontsize = 12)
axes[1].set_ylabel(r'value',fontsize = 12)
axes[1].legend()
axes[1].set_title('Hispanic')

plt.tight_layout()

fig.savefig('fairness_ah_dp.pdf')

### EQOPT

In [ ]:
k1_list = np.linspace(1,2,5)
_, theta_a_list = get_strat_optimal(u, alpha_asian, q, eps, a_a0, b_a0, (a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1, k1_list)
phi_func = np.vectorize(get_Phi, excluded = ['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k1','k2','k3'])
nonstrat_func = np.vectorize(non_strat_utility, excluded = ['u','alpha','a0','b0','a1','b1'])
utility_a_list = phi_func(u, alpha_asian, theta_a_list, q, eps, a_a0, b_a0, (a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1) + nonstrat_func(u, alpha_asian, theta_a_list, a_a0, b_a0,a_a1, b_a1)
utility_a_non = [get_Phi(u, alpha_asian, uhat_a, q, eps, a_a0, b_a0, (a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1) + non_strat_utility(u, alpha_asian, uhat_a, a_a0, b_a0,a_a1, b_a1)]*5
manip_list_a = get_manip_prob(theta_a_list,q,eps,(a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1)
cdf_a = np.array([fair_measure(theta, a_a0, b_a0, a_a1, b_a1, alpha_asian, 'eqopt') for theta in theta_a_list])
manip_hat_a = get_manip_prob(uhat_a,q,eps,(a_a0+a_a1)/2, (b_a0+b_a1)/2, a_a1, b_a1)
cdf_a_hat = fair_measure(uhat_a, a_a0, b_a0, a_a1, b_a1, alpha_asian, 'eqopt')

k1_list = np.linspace(1,2,5)
_, theta_h_list = get_strat_optimal(u, alpha_h, q, eps, a_h0, b_h0, (a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1, k1_list)
phi_func = np.vectorize(get_Phi, excluded = ['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k1','k2','k3'])
nonstrat_func = np.vectorize(non_strat_utility, excluded = ['u','alpha','a0','b0','a1','b1'])
utility_h_list = phi_func(u, alpha_h, theta_h_list, q, eps, a_h0, b_h0, (a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1) + nonstrat_func(u, alpha_h, theta_h_list, a_h0, b_h0,a_h1, b_h1)
utility_h_non = [get_Phi(u, alpha_h, uhat_h, q, eps, a_h0, b_h0, (a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1) + non_strat_utility(u, alpha_h, uhat_h, a_h0, b_h0,a_h1, b_h1)]*5
manip_list_h = get_manip_prob(theta_h_list,q,eps,(a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1)
cdf_h = np.array([fair_measure(theta, a_h0, b_h0, a_h1, b_h1, alpha_h, 'eqopt') for theta in theta_h_list])
manip_hat_h = get_manip_prob(uhat_h,q,eps,(a_h0+a_h1)/2, (b_h0+b_h1)/2, a_h1, b_h1)
cdf_h_hat = fair_measure(uhat_h, a_h0, b_h0, a_h1, b_h1, alpha_h, 'eqopt')

# plot optimal threshold, (non)-strategic utility, and manipulation probability
x = k1_list
fig, axes = plt.subplots(1,2)
fig.set_size_inches(7, 3)
axes[0].tick_params(axis='both', which='major', labelsize=14)
axes[1].tick_params(axis='both', which='major', labelsize=14)
axes[0].plot(x,manip_list_a, marker = 'o',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
axes[0].plot(x,utility_a_list,marker = 'o',label = r'strategic utility',linestyle = 'solid',color = 'blue')
axes[0].plot(x,cdf_a, marker = 'o',label = r'$F_{X|Y,S}(\theta|1,s)$',linestyle = 'dotted',color = 'green')
axes[0].plot(x, utility_a_non, marker = 'o',label = r'non-strategic utility',linestyle = 'solid',color = 'red')
axes[0].plot(x, [manip_hat_a]*5,marker = 'o',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
axes[0].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$',fontsize=12)
axes[0].set_ylabel(r'value',fontsize = 12)
axes[0].set_title('Asian')

axes[1].plot(x,manip_list_h, marker = 'o',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
axes[1].plot(x,utility_h_list,marker = 'o',label = r'strategic utility',linestyle = 'solid',color = 'blue')
axes[1].plot(x,cdf_h, marker = 'o',label = r'$F_{X|Y,S}(\theta|1,s)$',linestyle = 'dotted',color = 'green')
axes[1].plot(x, utility_h_non, marker= 'o', label = r'non-strategic utility',linestyle = 'solid',color = 'red')
axes[1].plot(x, [manip_hat_h]*5,marker = 'o',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
axes[1].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$',fontsize=12)
axes[1].set_ylabel(r'value',fontsize = 12)
axes[1].legend(fontsize=8)
axes[1].set_title('Hispanic')

plt.tight_layout()

fig.savefig('fairness_ah.pdf')


In [ ]:
k1_list = np.linspace(1,2,5)
_, theta_c_list = get_strat_optimal(u, alpha_c, q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1, k1_list)
phi_func = np.vectorize(get_Phi, excluded = ['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k1','k2','k3'])
nonstrat_func = np.vectorize(non_strat_utility, excluded = ['u','alpha','a0','b0','a1','b1'])
utility_c_list = phi_func(u, alpha_c, theta_c_list, q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1) + nonstrat_func(u, alpha_c, theta_c_list, a_c0, b_c0,a_c1, b_c1)
utility_c_non = [get_Phi(u, alpha_c, uhat_c, q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1) + non_strat_utility(u, alpha_c, uhat_c, a_c0, b_c0,a_c1, b_c1)]*5
manip_list_c = get_manip_prob(theta_c_list,q,eps,(a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
cdf_c = np.array([fair_measure(theta, a_c0, b_c0, a_c1, b_c1, alpha_c, 'eqopt') for theta in theta_c_list])
manip_hat_c = get_manip_prob(uhat_c,q,eps,(a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
cdf_c_hat = fair_measure(uhat_c, a_c0, b_c0, a_c1, b_c1, alpha_c, 'eqopt')


k1_list = np.linspace(1,2,5)
strat_func = np.vectorize(get_strat_optimal, excluded=['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k2', 'k3'])
_, theta_aa_list = get_strat_optimal(u, alpha_aa, q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1, k1_list)
phi_func = np.vectorize(get_Phi, excluded = ['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k1','k2','k3'])
nonstrat_func = np.vectorize(non_strat_utility, excluded = ['u','alpha','a0','b0','a1','b1'])
utility_aa_list = phi_func(u, alpha_aa, theta_aa_list, q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1) + nonstrat_func(u, alpha_aa, theta_aa_list, a_aa0, b_aa0,a_aa1, b_aa1)
utility_aa_non = [get_Phi(u, alpha_aa, uhat_aa, q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1) + non_strat_utility(u, alpha_aa, uhat_aa, a_aa0, b_aa0,a_aa1, b_aa1)]*5
manip_list_aa = get_manip_prob(theta_aa_list,q,eps,(a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
cdf_aa = np.array([fair_measure(theta, a_aa0, b_aa0, a_aa1, b_aa1, alpha_aa, 'eqopt') for theta in theta_aa_list])
manip_hat_aa = get_manip_prob(uhat_aa,q,eps,(a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
cdf_aa_hat = fair_measure(uhat_aa, a_aa0, b_aa0, a_aa1, b_aa1, alpha_aa, 'eqopt')


# plot optimal threshold, (non)-strategic utility, and manipulation probability
x = k1_list
fig, axes = plt.subplots(1,2)
fig.set_size_inches(7, 3)
axes[0].tick_params(axis='both', which='major', labelsize=14)
axes[1].tick_params(axis='both', which='major', labelsize=14)
axes[0].plot(x,manip_list_c, marker = 'o',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
axes[0].plot(x,utility_c_list,marker = 'o',label = r'strategic utility',linestyle = 'solid',color = 'blue')
axes[0].plot(x,cdf_c, marker = 'o',label = r'$F_{X|Y,S}(\theta|1,s)$',linestyle = 'dotted',color = 'green')
axes[0].plot(x, utility_c_non, marker = 'o', label = r'non-strategic utility',linestyle = 'solid',color = 'red')
axes[0].plot(x, [manip_hat_c]*5,marker = 'o',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
# axes[0].plot(x, [cdf_c_hat]*5,label = r'$F_{X|Y,S}(\widehat{theta}^*|1,s)$')
axes[0].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$', fontsize = 12)
axes[0].set_ylabel(r'value', fontsize = 12)
axes[0].set_title('Caucasian',fontsize = 12)

axes[1].plot(x,manip_list_aa, marker = 'o',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
axes[1].plot(x,utility_aa_list,marker = 'o',label = r'strategic utility',linestyle = 'solid',color = 'blue')
axes[1].plot(x,cdf_aa, marker = 'o',label = r'$F_{X|Y,S}(\theta|1,s)$',linestyle = 'dotted',color = 'green')
axes[1].plot(x, utility_aa_non ,marker = 'o',label = r'non-strategic utility',linestyle = 'solid',color = 'red')
axes[1].plot(x, [manip_hat_aa]*5,marker = 'o',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
# axes[1].plot(x, [cdf_aa_hat]*5,label = r'$F_{X|Y,S}(\widehat{\theta}^*|1,s)$')
axes[1].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$', fontsize = 12)
axes[1].set_ylabel(r'value', fontsize = 12)
axes[1].legend(fontsize = 9)
axes[1].set_title('African American',fontsize = 12)
plt.tight_layout()
fig.savefig('fairness_caa.pdf')

### Noisy

In [34]:
# vars not changed
def non_strat_utility(u,alpha,theta,a0,b0,a1,b1):
    U = (1 - beta.cdf(theta,a1,b1))*alpha*u - (1-beta.cdf(theta,a0,b0))*(1-alpha)*u  
    return U

sd = [0.05, 0.1]
def gen_delta(std, n = 10):
    return np.random.normal(0,std,n)
uhat_c = opt_non_strategic(u,alpha_c,a_c0,b_c0,a_c1,b_c1)[1]
uhat_aa = opt_non_strategic(u,alpha_aa,a_aa0,b_aa0,a_aa1,b_aa1)[1]
k1_list = np.linspace(1,2,5)
phi_func = np.vectorize(get_Phi, excluded = ['u', 'alpha', 'q', 'eps', 'a0', 'b0', 'ai', 'bi', 'a1', 'b1', 'k1','k2','k3'])
nonstrat_func = np.vectorize(non_strat_utility, excluded = ['u','alpha','a0','b0','a1','b1'])

In [ ]:
for i in range(len(sd)):
    print(i)
    deltas = gen_delta(sd[i])
    
    theta_c_list = np.zeros([10,5])
    utility_c_list = np.zeros([10,5])
    utility_c_non = np.zeros([10,5])
    manip_list_c = np.zeros([10,5])
    manip_hat_c = np.zeros(10)
    cdf_c = np.zeros([10,5])
    cdf_c_hat = np.zeros([10,5])

    theta_aa_list = np.zeros([10,5])
    utility_aa_list = np.zeros([10,5])
    utility_aa_non = np.zeros([10,5])
    manip_list_aa = np.zeros([10,5])
    manip_hat_aa = np.zeros(10)
    cdf_aa = np.zeros([10,5])
    cdf_aa_hat = np.zeros([10,5])
    
    for j in range(len(deltas)):
        print(j)
        _, theta_c_list[j] = get_strat_optimal(u, alpha_c, q+deltas[j], eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1, k1_list)
        utility_c_list[j] = phi_func(u, alpha_c, theta_c_list[j], q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1) + nonstrat_func(u, alpha_c, theta_c_list[j], a_c0, b_c0,a_c1, b_c1)
        utility_c_non[j] = [get_Phi(u, alpha_c, uhat_c, q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1) + non_strat_utility(u, alpha_c, uhat_c, a_c0, b_c0,a_c1, b_c1)]*5
        manip_list_c[j] = get_manip_prob(theta_c_list[j],q,eps,(a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
        cdf_c[j] = np.array([fair_measure(theta, a_c0, b_c0, a_c1, b_c1, alpha_c, 'eqopt') for theta in theta_c_list[j]])
        manip_hat_c[j] = get_manip_prob(uhat_c,q,eps,(a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
        cdf_c_hat[j] = fair_measure(uhat_c, a_c0, b_c0, a_c1, b_c1, alpha_c, 'eqopt')
        
        _, theta_aa_list[j] = get_strat_optimal(u, alpha_aa, q+deltas[j], eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1, k1_list)
        utility_aa_list[j] = phi_func(u, alpha_aa, theta_aa_list[j], q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1) + nonstrat_func(u, alpha_aa, theta_aa_list[j], a_aa0, b_aa0,a_aa1, b_aa1)
        utility_aa_non[j] = [get_Phi(u, alpha_aa, uhat_aa, q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1) + non_strat_utility(u, alpha_aa, uhat_aa, a_aa0, b_aa0,a_aa1, b_aa1)]*5
        manip_list_aa[j] = get_manip_prob(theta_aa_list[j],q,eps,(a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
        cdf_aa[j] = np.array([fair_measure(theta, a_aa0, b_aa0, a_aa1, b_aa1, alpha_aa, 'eqopt') for theta in theta_aa_list[j]])
        manip_hat_aa[j] = get_manip_prob(uhat_aa,q,eps,(a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
        cdf_aa_hat[j] = fair_measure(uhat_aa, a_aa0, b_aa0, a_aa1, b_aa1, alpha_aa, 'eqopt')
        
    error_c_list = theta_c_list.std(axis = 0)  
    theta_c_list = theta_c_list.mean(axis = 0)
    error_c_non = np.zeros(5)
    utility_c_non = utility_c_non.mean(axis=0)
    error_manip_c = manip_list_c.std(axis=0)
    manip_list_c = manip_list_c.mean(axis=0)
    error_hat_c = np.zeros(5)
    manip_hat_c = manip_hat_c.mean()
    error_cdf_c_hat = np.zeros(5)
    cdf_c_hat = cdf_c_hat.mean(axis=0)
    error_cdf_c = cdf_c.std(axis=0)
    cdf_c = cdf_c.mean(axis=0)
    error_utility_c = utility_c_list.std(axis=0)
    utility_c_list = utility_c_list.mean(axis=0)
    
    error_aa_list = theta_aa_list.std(axis = 0)
    theta_aa_list = theta_aa_list.mean(axis = 0)
    error_aa_non = np.zeros(5)
    utility_aa_non = utility_aa_non.mean(axis=0)
    error_manip_aa = manip_list_aa.std(axis=0)
    manip_list_aa = manip_list_aa.mean(axis=0)
    error_hat_aa = np.zeros(5)
    manip_hat_aa = manip_hat_aa.mean()
    error_cdf_aa_hat = np.zeros(5)
    cdf_aa_hat = cdf_aa_hat.mean(axis=0)
    error_cdf_aa = cdf_aa.std(axis=0)
    cdf_aa = cdf_aa.mean(axis=0)
    error_utility_aa = utility_aa_list.std(axis=0)
    utility_aa_list = utility_aa_list.mean(axis=0)
    
    print(f'average strategic utility for caucasian when sd = {sd[i]} and k1 = 1.25 is {utility_c_list[1]}')
    print(f'average Pm for caucasian when sd = {sd[i]} and k1 = 1 is {manip_list_c[0]}')
    print(f'average Pm for caucasian when sd = {sd[i]} and k1 = 1.25 is {manip_list_c[1]}')
    print(f'average cdf for caucasian when sd = {sd[i]} and k1 = 1 is {cdf_c[0]}')
    print(f'average cdf for caucasian when sd = {sd[i]} and k1 = 1.25 is {cdf_c[1]}')
    
    print(f'average strategic utility for aa when sd = {sd[i]} and k1 = 1.25 is {utility_aa_list[1]}')
    print(f'average Pm for aa when sd = {sd[i]} and k1 = 1 is {manip_list_aa[0]}')
    print(f'average Pm for aa when sd = {sd[i]} and k1 = 1.25 is {manip_list_aa[1]}')
    print(f'average cdf for aa when sd = {sd[i]} and k1 = 1 is {cdf_aa[0]}')
    print(f'average cdf for aa when sd = {sd[i]} and k1 = 1.25 is {cdf_aa[1]}')
    
    # plot optimal threshold, (non)-strategic utility, and manipulation probability
    x = k1_list
    fig, axes = plt.subplots(1,2)
    fig.set_size_inches(7, 3)
    fig.tight_layout(pad=2.0)
    axes[0].tick_params(axis='both', which='major', labelsize=14)
    axes[1].tick_params(axis='both', which='major', labelsize=14)
    axes[0].errorbar(x,manip_list_c,error_manip_c, marker = '.',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
    axes[0].errorbar(x,utility_c_list,error_utility_c,marker = '.',label = r'strategic utility',linestyle = 'solid',color = 'blue')
    axes[0].errorbar(x,cdf_c,error_cdf_c, marker = '.',label = r'$F_{X|Y,S}(\theta|1,s)$',linestyle = 'dotted',color = 'green')
    axes[0].errorbar(x, utility_c_non,error_c_non, marker = '.', label = r'non-strategic utility',linestyle = 'solid',color = 'red')
    axes[0].errorbar(x, [manip_hat_c]*5, error_hat_c, marker = '.',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
    # axes[0].plot(x, [cdf_c_hat]*5,label = r'$F_{X|Y,S}(\widehat{theta}^*|1,s)$')
    axes[0].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$', fontsize = 12)
    axes[0].set_ylabel(r'value', fontsize = 12)
    # axes[0].legend(fontsize = 7, bbox_to_anchor=(0.6, 0.47))
    axes[0].set_title('Caucasian',fontsize = 12)

    axes[1].errorbar(x,manip_list_aa,error_manip_aa, marker = '.',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
    axes[1].errorbar(x,utility_aa_list,error_utility_aa,marker = '.',label = r'strategic utility',linestyle = 'solid',color = 'blue')
    axes[1].errorbar(x,cdf_aa, error_cdf_aa,marker = '.',label = r'$F_{X|Y,S}(\theta|1,s)$',linestyle = 'dotted',color = 'green')
    axes[1].errorbar(x, utility_aa_non , error_aa_non, marker = '.',label = r'non-strategic utility',linestyle = 'solid',color = 'red')
    axes[1].errorbar(x, [manip_hat_aa]*5,error_hat_aa, marker = '.',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
    # axes[1].plot(x, [cdf_aa_hat]*5,label = r'$F_{X|Y,S}(\widehat{\theta}^*|1,s)$')
    axes[1].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$', fontsize = 12)
    axes[1].set_ylabel(r'value', fontsize = 12)
    axes[1].legend(fontsize = 8,bbox_to_anchor=(0.3, 0.42))
    axes[1].set_title('African American',fontsize = 12)
    
    fig.savefig(f'fairness_caa_q_noise_{i}.pdf')

In [ ]:
for i in range(len(sd)):
    print(i)
    deltas = gen_delta(sd[i])
    
    theta_c_list = np.zeros([10,5])
    utility_c_list = np.zeros([10,5])
    utility_c_non = np.zeros([10,5])
    manip_list_c = np.zeros([10,5])
    manip_hat_c = np.zeros(10)
    cdf_c = np.zeros([10,5])
    cdf_c_hat = np.zeros([10,5])

    theta_aa_list = np.zeros([10,5])
    utility_aa_list = np.zeros([10,5])
    utility_aa_non = np.zeros([10,5])
    manip_list_aa = np.zeros([10,5])
    manip_hat_aa = np.zeros(10)
    cdf_aa = np.zeros([10,5])
    cdf_aa_hat = np.zeros([10,5])
    
    for j in range(len(deltas)):
        print(j)
        _, theta_c_list[j] = get_strat_optimal(u, alpha_c, q, eps+deltas[j], a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1, k1_list)
        utility_c_list[j] = phi_func(u, alpha_c, theta_c_list[j], q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1) + nonstrat_func(u, alpha_c, theta_c_list[j], a_c0, b_c0,a_c1, b_c1)
        utility_c_non[j] = [get_Phi(u, alpha_c, uhat_c, q, eps, a_c0, b_c0, (a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1) + non_strat_utility(u, alpha_c, uhat_c, a_c0, b_c0,a_c1, b_c1)]*5
        manip_list_c[j] = get_manip_prob(theta_c_list[j],q,eps,(a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
        cdf_c[j] = np.array([fair_measure(theta, a_c0, b_c0, a_c1, b_c1, alpha_c, 'eqopt') for theta in theta_c_list[j]])
        manip_hat_c[j] = get_manip_prob(uhat_c,q,eps,(a_c0+a_c1)/2, (b_c0+b_c1)/2, a_c1, b_c1)
        cdf_c_hat[j] = fair_measure(uhat_c, a_c0, b_c0, a_c1, b_c1, alpha_c, 'eqopt')
        
        _, theta_aa_list[j] = get_strat_optimal(u, alpha_aa, q, eps+deltas[j], a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1, k1_list)
        utility_aa_list[j] = phi_func(u, alpha_aa, theta_aa_list[j], q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1) + nonstrat_func(u, alpha_aa, theta_aa_list[j], a_aa0, b_aa0,a_aa1, b_aa1)
        utility_aa_non[j] = [get_Phi(u, alpha_aa, uhat_aa, q, eps, a_aa0, b_aa0, (a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1) + non_strat_utility(u, alpha_aa, uhat_aa, a_aa0, b_aa0,a_aa1, b_aa1)]*5
        manip_list_aa[j] = get_manip_prob(theta_aa_list[j],q,eps,(a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
        cdf_aa[j] = np.array([fair_measure(theta, a_aa0, b_aa0, a_aa1, b_aa1, alpha_aa, 'eqopt') for theta in theta_aa_list[j]])
        manip_hat_aa[j] = get_manip_prob(uhat_aa,q,eps,(a_aa0+a_aa1)/2, (b_aa0+b_aa1)/2, a_aa1, b_aa1)
        cdf_aa_hat[j] = fair_measure(uhat_aa, a_aa0, b_aa0, a_aa1, b_aa1, alpha_aa, 'eqopt')
        
    error_c_list = theta_c_list.std(axis = 0)  
    theta_c_list = theta_c_list.mean(axis = 0)
    error_c_non = np.zeros(5)
    utility_c_non = utility_c_non.mean(axis=0)
    error_manip_c = manip_list_c.std(axis=0)
    manip_list_c = manip_list_c.mean(axis=0)
    error_hat_c = np.zeros(5)
    manip_hat_c = manip_hat_c.mean()
    error_cdf_c_hat = np.zeros(5)
    cdf_c_hat = cdf_c_hat.mean(axis=0)
    error_cdf_c = cdf_c.std(axis=0)
    cdf_c = cdf_c.mean(axis=0)
    error_utility_c = utility_c_list.std(axis=0)
    utility_c_list = utility_c_list.mean(axis=0)
    
    error_aa_list = theta_aa_list.std(axis = 0)
    theta_aa_list = theta_aa_list.mean(axis = 0)
    error_aa_non = np.zeros(5)
    utility_aa_non = utility_aa_non.mean(axis=0)
    error_manip_aa = manip_list_aa.std(axis=0)
    manip_list_aa = manip_list_aa.mean(axis=0)
    error_hat_aa = np.zeros(5)
    manip_hat_aa = manip_hat_aa.mean()
    error_cdf_aa_hat = np.zeros(5)
    cdf_aa_hat = cdf_aa_hat.mean(axis=0)
    error_cdf_aa = cdf_aa.std(axis=0)
    cdf_aa = cdf_aa.mean(axis=0)
    error_utility_aa = utility_aa_list.std(axis=0)
    utility_aa_list = utility_aa_list.mean(axis=0)
    
    print(f'average strategic utility for caucasian when sd = {sd[i]} and k1 = 1.25 is {utility_c_list[1]}')
    print(f'average Pm for caucasian when sd = {sd[i]} and k1 = 1 is {manip_list_c[0]}')
    print(f'average Pm for caucasian when sd = {sd[i]} and k1 = 1.25 is {manip_list_c[1]}')
    print(f'average cdf for caucasian when sd = {sd[i]} and k1 = 1 is {cdf_c[0]}')
    print(f'average cdf for caucasian when sd = {sd[i]} and k1 = 1.25 is {cdf_c[1]}')
    
    print(f'average strategic utility for aa when sd = {sd[i]} and k1 = 1.25 is {utility_aa_list[1]}')
    print(f'average Pm for aa when sd = {sd[i]} and k1 = 1 is {manip_list_aa[0]}')
    print(f'average Pm for aa when sd = {sd[i]} and k1 = 1.25 is {manip_list_aa[1]}')
    print(f'average cdf for aa when sd = {sd[i]} and k1 = 1 is {cdf_aa[0]}')
    print(f'average cdf for aa when sd = {sd[i]} and k1 = 1.25 is {cdf_aa[1]}')
    
    # plot optimal threshold, (non)-strategic utility, and manipulation probability
    x = k1_list
    fig, axes = plt.subplots(1,2)
    fig.set_size_inches(7, 3)
    fig.tight_layout(pad=2.0)
    axes[0].tick_params(axis='both', which='major', labelsize=14)
    axes[1].tick_params(axis='both', which='major', labelsize=14)
    axes[0].errorbar(x,manip_list_c,error_manip_c, marker = '.',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
    axes[0].errorbar(x,utility_c_list,error_utility_c,marker = '.',label = r'strategic utility',linestyle = 'solid',color = 'blue')
    axes[0].errorbar(x,cdf_c,error_cdf_c, marker = '.',label = r'$F_{X|Y,S}(\theta|1,s)$',linestyle = 'dotted',color = 'green')
    axes[0].errorbar(x, utility_c_non,error_c_non, marker = '.', label = r'non-strategic utility',linestyle = 'solid',color = 'red')
    axes[0].errorbar(x, [manip_hat_c]*5, error_hat_c, marker = '.',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
    # axes[0].plot(x, [cdf_c_hat]*5,label = r'$F_{X|Y,S}(\widehat{theta}^*|1,s)$')
    axes[0].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$', fontsize = 12)
    axes[0].set_ylabel(r'value', fontsize = 12)
    # axes[0].legend(fontsize = 7, bbox_to_anchor=(0.6, 0.47))
    axes[0].set_title('Caucasian',fontsize = 12)

    axes[1].errorbar(x,manip_list_aa,error_manip_aa, marker = '.',label = r'$P_M(\theta^*(k_1))$',linestyle = 'dotted',color = 'blue')
    axes[1].errorbar(x,utility_aa_list,error_utility_aa,marker = '.',label = r'strategic utility',linestyle = 'solid',color = 'blue')
    axes[1].errorbar(x,cdf_aa, error_cdf_aa,marker = '.',label = r'$F_{X|Y,S}(\theta|1,s)$',linestyle = 'dotted',color = 'green')
    axes[1].errorbar(x, utility_aa_non , error_aa_non, marker = '.',label = r'non-strategic utility',linestyle = 'solid',color = 'red')
    axes[1].errorbar(x, [manip_hat_aa]*5,error_hat_aa, marker = '.',label = r'$P_M(\widehat{\theta}^*)$',linestyle = 'dotted',color = 'red')
    # axes[1].plot(x, [cdf_aa_hat]*5,label = r'$F_{X|Y,S}(\widehat{\theta}^*|1,s)$')
    axes[1].set_xlabel(r'$k_1$ normalized by $u(1-\alpha)$', fontsize = 12)
    axes[1].set_ylabel(r'value', fontsize = 12)
    axes[1].legend(fontsize = 8,bbox_to_anchor=(0.3, 0.42))
    axes[1].set_title('African American',fontsize = 12)
    
    fig.savefig(f'fairness_caa_eps_noise_{i}.pdf')